# Data analysis project

### This is a final project for data analysis academy that I created with my colleague Rebeka Kopkas Uličná. Data had to annonymized so in some parts the code as well so this might look bit hectic.

- Importing a csv table of Czech energetic info and changed the column names to the second row (6 respondent categories, clusters)
- Renaming the first two columns as question and sub-question
- Using unidecode to get rid of the accents

In [ ]:
%pip install unidecode
import pandas as pd
from unidecode import unidecode
from pprint import pprint
import json
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import numpy as np
czech_df = pd.read_csv("../data/CZ_Energoprofily.csv", header=1)
czech_df.columns.values[0] = "CZ_dotaz" 
czech_df.columns.values[1] = "CZ_poddotaz"
czech_df.drop(columns = czech_df.columns[range(9,21)], inplace=True)
czech_df.columns = czech_df.columns.to_series().apply(unidecode)
czech_df.info()
czech_df.head()

- Getting rid of the question, using unidecode and using lambda function to skip Nan values

In [ ]:
czech_df['czdata1']=czech_df['CZ_dotaz'].apply(lambda value: unidecode(value) if pd.notnull(value) else value)
czech_df['czdata12']=czech_df['CZ_poddotaz'].apply(lambda value: unidecode(value) if pd.notnull(value) else value)
czech_df.head()

- Fill questions into all rows with answers (with ffil)
- Export first clustered dataset to csv

In [ ]:
czech_df["czdata1"] = czech_df["CZ_dotaz"].ffill()
czech_df.to_csv('soubor45.csv', sep=',', header=True, index=False, mode='w', encoding='UTF-8', decimal='.')

- One file contained coded questions and answers of 2000 + survey participants and 2 tables of "Labels" = codes and corresponding originals for the answers and questions in the first table. 
- Creating first level dictionary (questions), which translates labels on the "Data" dataset with the original questionaire's question. 

In [ ]:
questionaire_questions={}
labels_01=pd.read_csv("../data/labels_dict_01.csv", header=0)
# labels_01.info()
# Remove columns, which we don't need, remaining Name and Label
labels_01.drop(['Position', 'Measurement Level', 'Role', 'Width', 'Alignment', 'Print Format', 'Write Format',
       'Missing Values'], axis = 1, inplace=True)
# labels_01.columns
labels_01.set_index('Name', inplace=True) 
labels_01.head(5)
questionaire_questions = labels_01.to_dict(orient='dict')['Label']
pprint(questionaire_questions)

- Creating a second level dictionary (answers), which translates the labels on the "Data" table with the original answers. 
- Since the codes for the answers aren't unique, we had to retain the question code (and subsequently label) too. 
- Therefore we have created a nested dictionary structure

In [ ]:
questionaire_answers={} #nested dict
labels_02=pd.read_csv("../data/labels_dict_02.csv", header=0)
labels_02.columns.values[1] = "code_of_answer"
labels_02.columns.values[2] = "answer_lvl_2"
labels_02.columns
labels_02["Variable Value"] = labels_02['Variable Value'].ffill()
def create_dictionary(df):
    df.set_index("code_of_answer", inplace=True) #tohle dělá nastavení podle čeho budou keys
    return df.to_dict()["answer_lvl_2"]   #mrknout proč nám to padá s paramatrem index
apply(lambda x: x.set_index('code_of_answer').to_dict()["answer_lvl_2"])

- Export the two Answer and Question code dictionaries as JSON files. 
- Load our Data of 2000+ surveys.

In [ ]:
# SURVEY Data
data_CZ=pd.read_csv("../data/Data_CZ_energetika.csv", encoding='UTF-8')
data_CZ.info()
data_CZ.head()

### Data cleaning level 2:
- Codes for "I dont know / Don't want to tell" are: 99, '99[a]', '9[a]', 88, -9 in specific columns 
- this code show us what the exact names of our columns are 

In [ ]:
print(data_CZ.columns.tolist())

In [ ]:
data_CZ = data_CZ.replace({'vol21': {99: np.nan, 88: np.nan}})
data_CZ = data_CZ.replace({'vo1': {99: np.nan, 88: np.nan}})
data_CZ.loc[:, 'ident_1': 'ident_14'] = data_CZ.loc[:, 'ident_1':'ident_14'].replace('99', np.nan)
data_CZ.loc[:, 'czprob_1':'czprob_8'] = data_CZ.loc[:, 'czprob_1':'czprob_8'].replace('99', np.nan)
data_CZ.loc[:, 'mob2_1':'mob2_9'] = data_CZ.loc[:, 'mob2_1':'mob2_9'].replace('9', np.nan)
data_CZ.loc[:, ['gdf', 'dpri']] = data_CZ.loc[:, ['gdf', 'dpri']].replace(99, np.nan)
data_CZ.loc[:, 'heat2_1': 'heat2_6'] = data_CZ.loc[:, 'heat2_1': 'heat2_6'].replace(-9, np.nan)

Testing the results

In [ ]:
print(data_CZ.loc[11:15, 'ident_1':'ident_14'])
data_CZ.loc[:, 'mob2_1':'mob2_9'].info()
print(data_CZ['vol21'])

* Changing "Other" types of answers to the actual answers in the subsequent column

In [ ]:
pol_map = questionaire_answers

def replace_pol(row):
    if np.isnan(row["vo1"]):
        return np.nan
    if row["vo1"] == 666: 
        return row["v4"]
    return pol_map["V4"][str(int(row["vo1"]))]
data_CZ["vo1"] = data_CZ.apply(replace_pol, axis=1)

In [ ]:
pol_map = questionaire_answers 

def replace_pol(row):
    if np.isnan(row["vol21"]):
        return np.nan
    if row["vol21"] == 666: 
        return row["vol21_O_T_H"]
    if row["vol21"] == -9:
        return pol_map["E98"][str(row["el21"])]
    return pol_map["vo4"][str(row["vol21"])]

data_CZ["vol21"] = data_CZ.apply(replace_pol, axis=1)

In [ ]:
heating_map = questionaire_answers
def replace_heat(n):
    if n == 0:
        return np.nan
    return heating_map["Heat78"][str(int(n))]
data_CZ.loc[:, 'heat2_1': 'heat2_6'] = data_CZ.loc[:, 'heat2_1': 'heat2_6'].applymap(replace_heat, na_action="ignore")

- Replace codes with net income categories

In [ ]:
def replace_income(n):
    return questionaire_answers["Dprijem7"][str(int(n))]
data_CZ[['dpri']] = data_CZ[['dpri']].applymap(replace_income, na_action="ignore")


- Renaming the question columns (originally coded with abbreviations) using the questions dictionary
- All the answers (except for Nan values) are replaced by their string equivalents (as in the dictionaries) and subsequently replaced by their actual answers (using the answers dictionary)


- Exporting the dataframe of survey data into a csv with Nan values to visualize in PowerBI

In [ ]:
data_CZ.rename(columns=questionaire_questions, inplace=True)
data_CZ = data_CZ.applymap(str, na_action="ignore")
data_CZ.replace(questionaire_answers, inplace=True)
data_CZ.to_csv('Data_CZ_withNans2.csv', sep=',', header=True, index=False, mode='w', encoding='UTF-8', decimal='.')


- export both dictionaries to json files

In [ ]:
with open('questionaire_labels_answers.json', mode='w', encoding='utf-8') as fp:
    json.dump(questionaire_answers, fp, ensure_ascii=False, indent=4)
    
with open('questionaire_labels_questions.json', mode='w', encoding='utf-8') as fp:
    json.dump(questionaire_questions, fp, ensure_ascii=False, indent=4)

- Using csv reader and writer in order to be able to use questions with values such as "99[a]", "9[a]", 99, 88, "99", -9 - replacing it with none values. 

In [ ]:
not_valid_values = ["99[a]", "9[a]", 99, 88, "99", -9]
mob_questions = [
    "Otazka1",
    "Otazka2",
    "Otazka3",
    "Otazka4"
]
# questionaire_answers.to_csv('questionaire_labels_answers.csv', sep=',', header=True, index=False, mode='w', encoding='UTF-8', decimal='.')
with open('questionaire_labels_answers.csv', 'w', encoding="utf-8") as output:
    writer = csv.writer(output, delimiter=',')
    for session_id in sorted(questionaire_answers):
        questan = questionaire_answers[session_id]
        for item in questan:
            if session_id in mob_questions and item == "9":
                writer.writerow([session_id, None, questan[item]])
            elif item in not_valid_values:
                writer.writerow([session_id, None, questan[item]])
            else:
                writer.writerow([session_id, int(float(item)), questan[item]])

In [ ]:
with open('questionaire_labels_questions.csv', 'w', encoding="utf-8") as output:
    writer = csv.writer(output, delimiter=',')
    for column, question in questionaire_questions.items():
        writer.writerow([column, question])

* Create dimensions tables (74 of them) from questionaire labels answers, which will be used for answer labels sorting in Power BI. 

In [ ]:

data = pd.read_csv('questionaire_labels_answers.csv')
print(data.columns)
grouped = data.groupby('otazka78')

i=0

for group_name, group_data in grouped:
    output_data = group_data[['rozhodně nepředstavuje', '1']]
    output_data = output_data.rename(columns={'column3': group_name, 'column2': f'poradi_{group_name[:10]}'})
    output_data.to_csv(f'dimension_tables_Q_raw_CZ/{i}_{group_name[:10].replace("/", "_")}.csv', index=False)
    i+=1